# Flussbilanzanalyse mit U. maydis - Modell : Simulation und Verhältnisanalyse mit Glucose und Co-Feed von Acetat und Formiat zur CO2-negativen/neutralen Prozessgestaltung

In [ ]:
from os.path import join
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from cobra import Model, Reaction, Metabolite
from cobra.io import read_sbml_model

In [ ]:
ModelFile = join('..','model','iCL1079.xml')
model=read_sbml_model(ModelFile)

model.compartments

In [ ]:
model.reactions.query('h_e')
model.add_boundary(metabolite=model.metabolites.h_e, type='exchange', reaction_id='EX_h_e', lb=0, ub=0)

 ## Reactions

### acetate

In [ ]:
print('acetate reactions in cytoplasm')
for reaction in model.metabolites.ac_c.reactions:
    print(reaction.id, reaction, reaction.name)

print('')
print('acetate reactions in extracellular')
for reaction in model.metabolites.ac_e.reactions:
    print(reaction.id, reaction, reaction.name)    
# for reaction in model.metabolites.co2_c.reactions:
#     print(reaction.id, reaction, reaction.name)

# for reaction in model.metabolites.co2_e.reactions:
#     print(reaction.id, reaction, reaction.name)

### formate

In [ ]:
print('formate reactions in cytoplasm')
for reaction in model.metabolites.for_c.reactions:
    print(reaction.id, reaction, reaction.name)
    
    
print('')
print('formate reactions in extracellular')
for reaction in model.metabolites.for_e.reactions:
    print(reaction.id, reaction, reaction.name)

In [ ]:
model.summary()
# print(model.objective)
# model.optimize()

In [ ]:
medium=model.medium
with model:
    medium['EX_glc__D_e']= 5#7.69
#     medium['EX_ac_e']=10
    model.medium=medium
    solution=model.optimize()
    print(solution.fluxes['BIOMASS_REACTION'])
    print(model.summary())

In [ ]:
Glucose_list = np.array([5,7,9,11,13,15])
Acetate_list = np.array([0,2,4,6,8,10])
# medium_list = np.array([[x,y] for x in Glucose_list for y in Acetate_list])
Glc_list, Ace_list = np.meshgrid(Glucose_list, Acetate_list)
medium_list = np.transpose(np.reshape(np.array([Glc_list, Ace_list]),(2,-1)))
# print(medium_list)

In [ ]:
Bio_list = list()
for z in medium_list:
    x=z[0]
    y=z[1]
    with model:
        medium['EX_glc__D_e']=x
        medium['EX_ac_e']=y
        model.medium=medium
        solution=model.optimize()
        Bio_list.append(solution.fluxes['BIOMASS_REACTION'])
        
Bio_list = np.reshape(np.array(Bio_list),(-1,1))
Z = np.reshape(Bio_list,(6,6))
plt.contourf(Glc_list, Ace_list, Z, 20, cmap='coolwarm')
plt.colorbar(label='growth /h')
plt.xlabel('Glc in mmol/gCDW/h')
plt.ylabel('Ace in mmol/gCDW/h')
# plt.savefig('Umay-Glc-Ace_default.png')

## forcing proton export

In [ ]:
Bio_list = list()
for z in medium_list:
    x=z[0]
    y=z[1]
    with model:
        medium['EX_glc__D_e']=x
        medium['EX_ac_e']=y
        model.medium=medium
        model.reactions.EX_h_e.lower_bound = np.power(1.33,y)
        solution=model.optimize()
        Bio_list.append(solution.fluxes['BIOMASS_REACTION'])
        
Bio_list = np.reshape(np.array(Bio_list),(-1,1))
Z = np.reshape(Bio_list,(6,6))
plt.contourf(Glc_list, Ace_list, Z, 20, cmap='coolwarm')
plt.colorbar(label='growth /h')
plt.xlabel('Glc in mmol/gCDW/h')
plt.ylabel('Ace in mmol/gCDW/h')
# plt.savefig('Umay-Glc-Ace_InhHigh.png')